# 'Applied Data Science Capstone' Project

This notebook will mainly be used for the above-mentioned capstone project.

In [218]:
# importing libraries
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup as soup

!pip install geocoder
import geocoder

In [13]:
# Print the following the statement: Hello Capstone Project Course!
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


---

## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

1. Start by creating a new Notebook for this assignment.

In [16]:
# You are reading it right now, so this obviously already happened.

---

2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

![alt text](https://bit.ly/3pF2zYc "Screenshot of pandas dataframe")


3. To create the above dataframe:
  + The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
  + Only process the cells that have an assigned borough. Ignore cells with a borough that is **Not assigned**.
  + More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that **M5A** is listed twice and has two neighborhoods: **Harbourfront** and **Regent Park**. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in **row 11**  in the above table.
  + If a cell has a borough but a **Not assigned** neighborhood, then the neighborhood will be the same as the borough.
  + Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
  + In the last cell of your notebook, use the **.shape** method to print the number of rows of your dataframe.

In [253]:
# ----------------
# SCRAPING OF DATA
# ----------------

# define url
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# request
r = requests.Session()
response = r.get(url, timeout=10)  # expected value: 200

# using BeautifulSoup package for scraping webpages
soup = BeautifulSoup(response.content, 'html.parser')
pretty_soup = soup.prettify()

# find the table in the html
table=soup.find('table')

# Number of columns in the table
for row in right_table.findAll("tr"):
    columns = row.findAll("td")
len(columns)

# Number of rows in the table including header
rows = right_table.findAll("tr")
len(rows)

print("The table has a size of " + str(len(columns)), "columns and " + str(len(rows)-1), "rows (not including the header).")

The table has a size of 3 columns and 180 rows (not including the header).


In [254]:
# ----------------------
# CREATING THE DATAFRAME
# ----------------------

# getting column names / header
header = [th.text.rstrip() for th in rows[0].find_all("th")]  # optional: print(header)

# getting first row information as a test
row1 = [td.text.rstrip() for td in rows[1].find_all("td")]  # optional: print(row1)

# define empty list to fill with row information
allrows = []

# getting all rows now
for row in rows[1:]:
    data = [d.text.rstrip() for d in row.find_all('td')]
    allrows.append(data)  # optional: print(allrows)

# creating the DataFrame
df = pd.DataFrame(allrows)
df.head()

,0,1,2
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [255]:
# -------------------
# DATA PRE-PROCESSING
# -------------------

# assigning header to dataframe
df.columns = header

# deleting cells with a borough that is "Not assigned".
df.replace('Not assigned', np.NaN, inplace=True)
df.dropna(subset=["Borough"], axis=0, inplace=True)

# setting Neighbourhoods that are "Not assigned" to the same value as the Borough
df["Neighbourhood"].replace(np.NaN, "Borough", inplace=True)

# resetting the index
df.reset_index(drop=True, inplace=True)

# having a glimpse
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [205]:
# In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
print("df.shape =", df.shape,
      "--> so that means our DataFrame \"df\" consists of", df.shape[0], "rows and", df.shape[1], "columns.")

df.shape = (103, 3) --> so that means our DataFrame "df" consists of 103 rows and 3 columns.


---

4. Submit a link to your Notebook on your Github repository. **(10 marks)**

In [211]:
## submit a link to the new Notebook on your Github repository
print("https://github.com/ZeLilFish/Coursera_Capstone/blob/master/Segmenting%20and%20Clustering%20Neighborhoods%20in%20Toronto.ipynb")

https://github.com/ZeLilFish/Coursera_Capstone/blob/master/Segmenting%20and%20Clustering%20Neighborhoods%20in%20Toronto.ipynb


*Note: There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use pandas  to read the table into a pandas dataframe.*

*Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/*

*Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.*

---

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. 

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking  postal code **M5G** as an example, your code would look something like this:

```
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
```

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

Use the Geocoder package or the csv file to create the following dataframe:

![alt text](https://bit.ly/2ZFUGXI "Screenshot of pandas dataframe")

**Important Note:** There is a limit on how many times you can call geocoder.google function. It is 2500 times per day. This should be way more than enough for you to get acquainted with the package and to use it to get the geographical coordinates of the neighborhoods in the Toronto.

In [252]:
# ----------------------------------------------------
# GETTING LATITUDE & LONGITUDE FOR EVERY NEIGHBOURHOOD
# ----------------------------------------------------

# downloading the provided csv file
!wget -q -O "geospatial_data.csv" http://cocl.us/Geospatial_data

In [258]:
# reading csv into a DataFrame
df_geo = pd.read_csv("geospatial_data.csv")

# sorting both DataFrames by Postal Code
df_geo.sort_values(by=["Postal Code"], inplace=True)
df.sort_values(by=["Postal Code"], inplace=True)

# combining the two DataFrames into one
df_new = pd.merge(df, df_geo, on="Postal Code")
df_new

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


Once you are able to create the above dataframe, submit a link to the new Notebook on your Github repository. **(2 marks)**

**Note:** While including the link do not copy paste the URL. Use the embedded link option in the formatting  tools of the Response field to include the link. Check the  displayed in image below

In [ ]:
## submit a link to the new Notebook on your Github repository


---

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you. 

Just make sure:

1. to add enough Markdown cells to explain what you decided to do and to report any observations you make. 
2. to generate maps to visualize your neighborhoods and how they cluster together. 

Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. **(3 marks)**

In [ ]:
## submit a link to the new Notebook on your Github repository
